# Build Agent using LangChain and Neo4j Knowledge Graph
This document provides an overview of integrating a Neo4j graph database with LangChain, a framework for developing applications powered by language models.

## Prerequisites
Before you begin, ensure you have the following:
* A Neo4j database instance (e.g., Neo4j Aura). We use https://neo4j.io/, hosted in cloud.
* Access to Anthropic Claude. We use https://claude.ai, hosted in cloud.

![neo4j-graph](./example_data/neo4j-graph.png)

# Setting Up
Uncomment to install the package

In [24]:
# pip install -U neo4j langchain-anthropic

Uncomment if API key is not added yet

In [25]:
# import getpass
# import os

# os.environ["ANTHROPIC_API_KEY"] = getpass.getpass()
# password = getpass.getpass()

## Connecting to Neo4j
To connect to your Neo4j database, create an instance of the Neo4jGraph class from the langchain_community.graphs module:

In [26]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(url="neo4j+s://a7db75cc.databases.neo4j.io", username="neo4j", password=password)

# Populating the Graph

You can populate the graph by executing Cypher queries using the query method of the Neo4jGraph instance. This query creates customer nodes and establishes relationships with product and category nodes.

In [30]:
response = graph.query(
    """
// Merge customers into the graph
MERGE (c1:Customer {name: "John Doe", email: "johndoe@email.com"})
MERGE (c2:Customer {name: "Jane Smith", email: "janesmith@email.com"})

// Create a list of products and categories
WITH c1, c2, 
     [["Macbook Pro", "Electronics"], ["The Alchemist", "Books"], ["Nike Air Force", "Clothing"], ["Xbox Series X", "Electronics"]] AS products
     
// Create product nodes and category relationships
UNWIND products AS product_category
MERGE (p:Product {name: product_category[0]})
MERGE (cat:Category {name: product_category[1]})
MERGE (p)-[:BELONGS_TO]->(cat)

// Create purchase relationships for John Doe
MERGE (c1)-[:PURCHASED {date: date("2024-08-10"), quantity: 2}]->(p)
WITH c1, c2, p

// Create purchase relationships for Jane Smith with different dates and quantities
MERGE (c2)-[:PURCHASED {date: date("2024-09-01"), quantity: 1}]->(p)
"""
)
print(response)

[]


## Refreshing the Schema
After populating the graph, refresh the schema to ensure LangChain has access to the updated graph structure:

In [31]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Customer {name: STRING, email: STRING}
Product {name: STRING}
Category {name: STRING}
Relationship properties:
PURCHASED {date: DATE, quantity: INTEGER}
The relationships:
(:Customer)-[:PURCHASED]->(:Product)
(:Product)-[:BELONGS_TO]->(:Category)


## Creating a chain using GraphCypherQAChain
To query the graph using natural language, create an instance of the `GraphCypherQAChain` class. The chain generates a Cypher query based on the question, executes it against the graph, and returns the result. You can now query the graph using natural language.

In [32]:
from langchain.chains import GraphCypherQAChain
from langchain_anthropic import ChatAnthropic

chain = GraphCypherQAChain.from_llm(
    ChatAnthropic(model_name="claude-3-5-sonnet-20240620"), graph=graph, verbose=True
)

response = chain.invoke({"query": "What products did John Doe buy?"})
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Customer {name: 'John Doe'})-[p:PURCHASED]->(prod:Product)
RETURN prod.name, p.date, p.quantity
Full Context:
[{'prod.name': 'Macbook Pro', 'p.date': neo4j.time.Date(2024, 8, 10), 'p.quantity': 2}, {'prod.name': 'The Alchemist', 'p.date': neo4j.time.Date(2024, 8, 10), 'p.quantity': 2}, {'prod.name': 'Nike Air Force', 'p.date': neo4j.time.Date(2024, 8, 10), 'p.quantity': 2}, {'prod.name': 'Xbox Series X', 'p.date': neo4j.time.Date(2024, 8, 10), 'p.quantity': 2}]

> Finished chain.
{'query': 'What products did John Doe buy?', 'result': 'John Doe purchased several items on August 10, 2024. The products he bought include 2 Macbook Pro laptops, 2 copies of the book "The Alchemist", 2 pairs of Nike Air Force shoes, and 2 Xbox Series X gaming consoles.'}


In [38]:
response = chain.invoke({"query": "Which categories do Jane Smith's purchases belong to?"})
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Customer {name: 'Jane Smith'})-[:PURCHASED]->(:Product)-[:BELONGS_TO]->(cat:Category)
RETURN DISTINCT cat.name
Full Context:
[{'cat.name': 'Electronics'}, {'cat.name': 'Books'}, {'cat.name': 'Clothing'}]

> Finished chain.
{'query': "Which categories do Jane Smith's purchases belong to?", 'result': "Jane Smith's purchases belong to Electronics, Books, and Clothing categories."}
